# Cross-validation of DeeSse simulations

We have a synthetic dataset with 100 facies observations (encoded 0 or 1). We are going to use cross-validation approach to find the most suitable training image among three candidate TIs (we will call them A, B, C). We will also identify the what number of nearest neighbors should be used in order to get a best match with the data.

We will use 5-fold cross-validation as implemented in the scikit-learn library. DeesseEstimator class has been implemented in a way that it is a valid estimator object and can be supplied to model_selection tools of scikit-learn. Quadratic scoring rule is also implemented in the geone package. We are going to use 5-fold stratified cross-validation with data shuffling and grid search cross-validation to check all combinations of parameters.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.dummy import DummyClassifier

from geone.img import readImageGslib
from geone.gslib import read
from geone.imgplot import drawImage2D
from geone.deesseinterface import DeesseEstimator
from geone.cv_metrics import brier_score, zero_one_score, balanced_linear_score, SkillScore

In [ ]:
DATA_DIR = 'cv_data/'

## Observations

We have a classical gslib file with a point set and facies observations. We can load it to a dictionary of arrays, and then conveniently to a pandas dataframe.

In [ ]:
# read file to a dictionary of arrays
observations = read(DATA_DIR+'sample_100.gslib')

# and pass it directly to DataFrame constructor
df_observations = pd.DataFrame(observations)
df_observations.head()

In [ ]:
# we can check basic stats of the point set
df_observations.describe()

In [ ]:
# quick plot of the set
df_observations.plot.scatter(x='X', y='Y', c='code_real00000', cmap='cividis')

## DeeSse simulation engine

We want to test what DeeSse parameters (including the TI) fit the best our observation set.

In [ ]:
# Load training images using geone
ti_A = readImageGslib(DATA_DIR+'A.gslib')
ti_B = readImageGslib(DATA_DIR+'B.gslib')
ti_C = readImageGslib(DATA_DIR+'C.gslib')

In [ ]:
# draw training images using geone
drawImage2D(ti_A, categ=True)

In [ ]:
drawImage2D(ti_B, categ=True)

In [ ]:
drawImage2D(ti_C, categ=True)

We need to set up DeesseEstimator. It takes the same arguements as DeesseInput plus varnames, which denotes variable names which will be attached to the observation set. Each column should have a name (including coordinates). In our case we have X,Y,Z and 'facies'. 'facies' is also the simulated variable. Note that the names which were read to the pandas DataFrame in the previous step do not matter, as only the values are passed to the simulation engine. Therefore we need to attach names to them to have them properly converted (internally) to a conditioning set.

In [ ]:
# Define the interface for the DeeSse simulation tool
deesse_estimator = DeesseEstimator(
    varnames = ['X','Y','Z', 'facies'],
        nx=100, ny=100, nz=1,     # dimension of the simulation grid (number of cells)
        sx=1.0, sy=1.0, sz=1.0,   # cells units in the simulation grid (here are the default values)
        ox=0.0, oy=0.0, oz=0.0,   # origin of the simulation grid (here are the default values)
        nv=1, varname='facies',   # number of variable(s), name of the variable(s)
        nTI=1, TI=ti_A,           # number of TI(s), TI (class dsi.Img)
        distanceType=0,           # distance type: proportion of mismatching nodes (categorical var., default)
        nneighboringNode=20,      # max. number of neighbors (for the patterns)
        distanceThreshold=0.1,    # acceptation threshold (for distance between patterns)
        maxScanFraction=0.25,     # max. scanned fraction of the TI (for simulation of each cell)
        npostProcessingPathMax=1, # number of post-processing path(s)
        seed=20191201,            # seed (initialization of the random number generator)
        nrealization=4,           # number of realization(s)
        nthreads=4)               # number of threads to use for simulations

We can check if we properly defined the estimator by running simulate() method

In [ ]:
# run DeeSse and plot results
sim = deesse_estimator.simulate()
drawImage2D(sim['sim'][0], categ=True)

## Cross-validation engine

Now we are going to set-up cross validation. We will use scorers implemented in geone.cv_metrics module. Then, we will use GridSearchCV method of scikit-learn to evaluate all parameters combinations. We want to make sure that we use stratified cross-validation with data shuffling, therefore we are going to specify explicitly the split (again, by means of scikit-learn's splitter object).

Note that we could also use different model_selection tools available form scikit-learn, such as cross_validate, etc.

### Scoring functions

Scoring functions are implemented in geone. One could also define custom scorers or use functions from scikit-learn but the are not really suitable for probabilistic forecasts.

In [ ]:
# use Brier score (quadratic score), zero-one score, balanced linear score
# and quadratic skill, zero-one skill scores
scoring = {
    'brier':brier_score,
    'zero_one':zero_one_score,
    'linear':balanced_linear_score,
    'skill_brier':SkillScore(DummyClassifier(strategy='prior'), 0, brier_score),
    'skill_zero_one':SkillScore(DummyClassifier(strategy='prior'), 1, zero_one_score),
}

### Splitter definition

We want to use 5-fold stratified cross-validation on randomly shuffled data. We use splitter implemented in scikit-learn package.

In [ ]:
# Stratified 5-fold cross-validation with randomly shuffled data
cv = StratifiedKFold(n_splits=5,
                     shuffle=True,
                     random_state=20191201,
                    )


### Model selection tool

We are going to evaluate different sets of parameters, therefore it is convenient to use scikit learns's GridSearchCV.

In [ ]:
# evaluate 3 training images and 3 different numbers of neighbors
# cross_validator will need to run 9 times DeeSse (simulating nrealizations every time)
cross_validator =  GridSearchCV(deesse_estimator,
                    param_grid={'TI': [ti_A, ti_B, ti_C],
                               'nneighboringNode': [5, 10, 15]},
                    scoring=scoring,
                    n_jobs=1,
                    cv=cv,
                    refit=False,
                    verbose=0,
                    error_score='raise',
                    return_train_score=False,
                   )

In [ ]:
# run the cross-validation with our observed data
cross_validator.fit(X=df_observations[['X', 'Y', 'Z']],
                    y=df_observations['code_real00000'])

In [ ]:
# get the results as pandas DataFrame
results = pd.DataFrame(cross_validator.cv_results_)
results.head(5)

In [ ]:
# show three best scores (Brier score)
features = ['param_TI', 'param_nneighboringNode',
            'mean_test_brier', 'mean_test_zero_one',
            'mean_test_linear', 'mean_test_skill_brier', 'mean_test_skill_zero_one']
results.sort_values(by='mean_test_brier', ascending=False).head(3)[features]

The training image A has a higher quadratic score and also zero-one score. Therefore, it is probably a more suitable training image for the problem at hand. It is also better to use a bigger number of neighboring nodes.